In [1]:
import pandas as pd
import numpy as np
from itertools import combinations_with_replacement 
from time import sleep

In [2]:
import urllib.request
import json

# Your Bing Maps Key 
bingMapsKey = "AhV9mA0UagDnVW1_-j1qcw8_Vm-W7s5lsBZhF9KDhrzVIBXXiAEDfH6MAPv9X3Kh" #my bing API key

In [3]:
node_df = pd.read_csv(r'node_zip_cap.csv',converters={i: str for i in range(165)}) #keeps the 0 in front of 4 digit zips
# pathID_df = pd.read_csv(r'PathID_demand.csv')

In [4]:
node_df.head() #should have 165 total nodes len(node_df['ZIP'])

,First Origin,ZIP,Capacity
0,11,14219,4000
1,16,13601,7000
2,17,06457,7000
3,18,01887,7000
4,20,13209,4000


In [5]:
comb = combinations_with_replacement(node_df['ZIP'], 2) 

In [6]:
comb_list = []
for i in comb:
    comb_list.append(i)

In [7]:
len(comb_list) # should be 13695

13695

In [8]:
comb_set = set(comb_list)
comb_list = list(comb_set)

In [9]:
len(comb_list) #should be 3012

3012

In [10]:
df_dist = pd.DataFrame(index=np.arange(0, len(comb_list)),columns = ('Origin', 'Destination', 'Distance', 'Time'))

In [11]:
df_dist.head(10)

,Origin,Destination,Distance,Time
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [12]:
for i in range(len(comb_list)):
    try:
        Origin = comb_list[i][0]
        Destination = comb_list[i][1]
        
        df_dist.at[i, 'Origin']=Origin
        df_dist.at[i, 'Destination']=Destination
        
        
        encodedOrg = urllib.parse.quote(Origin, safe='')
        encodedDest = urllib.parse.quote(Destination, safe='')
        
        routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Driving?wp.0=" + encodedOrg + "&wp.1=" + encodedDest +"distanceUnit={Km}&optimize=time&key=" + bingMapsKey
    
        request = urllib.request.Request(routeUrl)
        response = urllib.request.urlopen(request)
        r = response.read().decode(encoding="utf-8")
        result = json.loads(r)
        
        df_dist.at[i, 'Distance']= round(int(result["resourceSets"][0]["resources"][0]["travelDistance"])/1.60934)
        df_dist.at[i, 'Time']= round(result["resourceSets"][0]["resources"][0]["travelDuration"]/3600)

    except:
        try:
            routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Driving?wp.0=" + encodedDest + "&wp.1=" + encodedOrg +"distanceUnit={Km}&optimize=time&key=" + bingMapsKey
        
            request = urllib.request.Request(routeUrl)
            response = urllib.request.urlopen(request)
            r = response.read().decode(encoding="utf-8")
            result = json.loads(r)
            
            df_dist.at[i, 'Distance']= round(int(result["resourceSets"][0]["resources"][0]["travelDistance"])/1.60934)
            df_dist.at[i, 'Time']= round(result["resourceSets"][0]["resources"][0]["travelDuration"]/3600)
        except:
            pass
    print((i/(len(comb_list)+1))*100,'\r',end="")

In [13]:
df_dist.head(100)

,Origin,Destination,Distance,Time
0,05452,07307,300,5
1,07801,08810,58,1
2,06279,11741,170,3
3,08817,08832,7,0
4,07606,17057,183,3
5,13601,13748,149,2
6,03053,19341,362,6
7,05452,14586,319,6
8,07307,07606,NaN,NaN
9,02554,13209,406,9


In [14]:
df_dist.to_csv(r'distance_data.csv') #fill in the NANs by swapping the API reasds for dist/origin, should still work

In [15]:
# import urllib.request
# import json

# # Your Bing Maps Key 
# bingMapsKey = "AhV9mA0UagDnVW1_-j1qcw8_Vm-W7s5lsBZhF9KDhrzVIBXXiAEDfH6MAPv9X3Kh"

# input information
# longitude = -122.019943
# latitude = 37.285989
Origin = "17339"
Destination = "14219"

encodedOrg = urllib.parse.quote(Origin, safe='')
encodedDest = urllib.parse.quote(Destination, safe='')

routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Driving?wp.0=" + encodedOrg + "&wp.1=" + encodedDest +"distanceUnit={Km}&optimize=time&key=" + bingMapsKey

request = urllib.request.Request(routeUrl)
response = urllib.request.urlopen(request)

r = response.read().decode(encoding="utf-8")
result = json.loads(r)



print(round(int(result["resourceSets"][0]["resources"][0]["travelDistance"])/1.60934)) #travel distance in miles
print(round(result["resourceSets"][0]["resources"][0]["travelDuration"]/3600)) # travel time in hours


298
5


In [16]:
for i in range(1):
    try:
        Origin = "16063"
        Destination = "17339"
        
        encodedOrg = urllib.parse.quote(Origin, safe='')
        encodedDest = urllib.parse.quote(Destination, safe='')
        
        routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Driving?wp.0=" + encodedOrg + "&wp.1=" + encodedDest +"distanceUnit={Km}&optimize=time&key=" + bingMapsKey
        
        request = urllib.request.Request(routeUrl)
        response = urllib.request.urlopen(request)
        
        r = response.read().decode(encoding="utf-8")
        result = json.loads(r)
        
        
        
        print(round(int(result["resourceSets"][0]["resources"][0]["travelDistance"])/1.60934)) #travel distance in miles
        print(round(result["resourceSets"][0]["resources"][0]["travelDuration"]/3600)) # travel time in hours
        
    except:
        routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Driving?wp.0=" + encodedDest + "&wp.1=" +encodedOrg +"distanceUnit={Km}&optimize=time&key=" + bingMapsKey
        
        request = urllib.request.Request(routeUrl)
        response = urllib.request.urlopen(request)
        
        r = response.read().decode(encoding="utf-8")
        result = json.loads(r)
        
        
        
        print(round(int(result["resourceSets"][0]["resources"][0]["travelDistance"])/1.60934)) #travel distance in miles
        print(round(result["resourceSets"][0]["resources"][0]["travelDuration"]/3600)) # travel time in hours
    finally:
        pass

232
3
